In [ ]:
# --- repo bootstrap ---------------------------------------------------------
from pathlib import Path
from dotenv import load_dotenv
import os, sys
import subprocess, sys, importlib, os, re
from datetime import datetime
import truthbrush as tb

def repo_root(start: Path) -> Path:
    cur = start.resolve()
    while cur != cur.parent:
        if (cur / ".env").exists() or (cur / ".git").exists():
            return cur
        cur = cur.parent
    raise RuntimeError("repo root not found")

ROOT = repo_root(Path.cwd())
load_dotenv(ROOT / ".env")             # loads secrets
sys.path.append(str(ROOT / "src"))     # optional helpers

DATA_DIR = ROOT / "data"
OUT_DIR  = ROOT / "outputs"
FIG_DIR  = OUT_DIR / "figs"; FIG_DIR.mkdir(exist_ok=True)

print("Repo root:", ROOT)

In [ ]:
# ────────── UNIVERSAL PATCH CELL  (run once, very top of notebook) ──────────
import subprocess, sys, importlib, os, types
from pathlib import Path

# 1️⃣  make sure both python-dotenv and curl_cffi exist
def ensure(pkg, src=None):
    try:
        importlib.import_module(pkg)
    except ModuleNotFoundError:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "--quiet", src or pkg]
        )

ensure("python-dotenv")
ensure("curl_cffi")

# 2️⃣  reload .env (override=True guarantees fresh values)
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(usecwd=True), override=True)

# 3️⃣  import truthbrush and inject curl_cffi so NameError can’t happen
import curl_cffi                     # noqa:  F401  (needed for side-effect)
import truthbrush.api as tb_api
tb_api.curl_cffi = curl_cffi         # hand it to truthbrush’s module scope

import truthbrush as tb
print("✔ Patch cell finished – environment refreshed, curl_cffi wired\n")


## Score with Anthropic Model of Choice

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════╗
# ║  SCORE Truth Social posts - Production Version                        ║
# ╚══════════════════════════════════════════════════════════════════════╝
from pathlib import Path
import json, time, pandas as pd, tqdm, re
import anthropic
from dotenv import load_dotenv
import os
import requests
import logging

load_dotenv()

# Configuration - can change model
ROOT = Path.cwd().resolve().parents[0]
TRUTH_CSV = ROOT / "outputs" / "truth_matches_merged.csv"
OUT_CSV = ROOT / "outputs" / "truth_scores_anthropic_claude-opus-4-20250514.csv"
MODEL = "claude-opus-4-20250514"

# Gold set
GOLD_EX = """
```
# Real-post EXEMPLARS (post → E,B,C):

I never thought America could be talking nuclear war… our leaders’ rhetoric…                              → 10,0,0
“World War III anyone?”                                                                                    → 10,-1,0
FIRST COME THE TANKS, THEN COME THE NUKES. Get this crazy war ended, NOW.                                  → 5,-1,1
Sharing Trump-peace article (Biden escalates war)                                                          → 0,0,0
NATO condemns Putin for nuclear threats…                                                                   → 9,1,0
Biden dragging us toward World War III—he must stop.                                                       → 10,0,1
Call for GOP to use the 25th Amendment over Ukraine policy.                                                → 1,0,1
Why won’t Europe pay its share for Ukraine defence?                                                        → 1,0,0
Biden’s failures everywhere… same with Russia/Ukraine, maybe worse.                                        → 1,0,0
VOTE FOR TRUMP – MAGA2024!                                                                                 → 1,0,1
Russia threatens nukes; we have no negotiators—NO GOOD.                                                    → 9,1,0
“Look at the missiles… NONE OF THIS HAPPENED WHILE I WAS PRESIDENT!”                                       → 7,0,0
Just landed in Rome to help END this cruel and senseless war.                                              → 0,-1,0
Trump statement: negotiations should begin immediately.                                                    → 0,-1,0
Why give $40 B to Ukraine while Europe does so little?                                                     → 4,0,0
Kherson village liberated—donate to 🇺🇦 aid funds today!                                                    → 2,1,1
Avdeyevka update: fighting continues, no big changes.                                                      → 3,-1,0
Biden wants talks with Xi & Putin re: possible WW III—he “doesn't remember quitting…”                      → 8,0,0
```
"""


# Prompt
TRUTH_PROMPT = """You are analyzing Truth Social posts about the Russia-Ukraine war.
Score each post on THREE dimensions:
1. Escalation (0-10): How militarily escalatory is the post?
2. Blame (-1, 0, 1): Who does the post blame?
   - 0 = Blames Ukraine/NATO/West
   - 1 = Blames Russia/Putin
   - -1 = No clear blame
3. Call-to-Action (0, 1): Does it urge concrete action?
   - 1 = Yes (vote, donate, stop funding, etc.)
   - 0 = No explicit call

Reference scale for escalation:
0: Humanitarian/diplomatic content
1: Peace negotiation focus
2: Political warfare/disinformation
3: Routine warfare casualties
4: Major aid package discussions
5: Major weapons systems
6: Advanced weapons escalation
7: Nation-wide strikes
8: Strategic threats near borders
9: Nuclear weapons threats
10: Direct nuclear war rhetoric

{GOLD_EX}

CRITICAL: Respond ONLY with three integers in format E,B,C
No spaces, no decimals, no words - just three numbers with two commas.
Example: 5,0,1"""

# Load data
df = pd.read_csv(TRUTH_CSV, parse_dates=["created_at"])
print(f"📊 Processing {len(df)} Truth Social posts")

# Initialize client
client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

# ── quiet the httpx + anthropic client ────────────────────────────────
for name in ("httpx", "anthropic"):
    logging.getLogger(name).setLevel(logging.WARNING)   # or logging.ERROR

# FULL RUN - Comment out the next two lines for dry run
# df = df.head(5).copy()
# print(f"🧪 DRY RUN WITH {len(df)} POSTS")

# Add index for tracking
df["batch_idx"] = range(len(df))

# Prepare batch requests
requests_list = []
for idx, row in df.iterrows():
    if pd.isna(row.get("text")) or str(row["text"]).strip() == "":
        continue
        
    request = {
        "custom_id": str(row["batch_idx"]),
        "params": {
            "model": MODEL,
            "max_tokens": 25,  # Increased slightly for safety
            "temperature": 0,
            "system": TRUTH_PROMPT,
            "messages": [
                {"role": "user", "content": str(row["text"])[:1500]}
            ]
        }
    }
    requests_list.append(request)

print(f"📝 Prepared {len(requests_list)} requests")

# Create batch
batch = client.messages.batches.create(requests=requests_list)
print(f"🚀 Launched batch {batch.id}")

# Monitor progress
bar = tqdm.tqdm(total=len(requests_list), desc="Processing", unit="req")
start_time = time.time()
while True:
    batch_status = client.messages.batches.retrieve(batch.id)
    completed = (batch_status.request_counts.succeeded + 
                batch_status.request_counts.errored + 
                batch_status.request_counts.canceled + 
                batch_status.request_counts.expired)
    bar.n = completed
    bar.refresh()
    
    if batch_status.processing_status == "ended":
        bar.close()
        break
    
    time.sleep(5)

elapsed_time = time.time() - start_time
print(f"✅ Batch processing complete in {elapsed_time/60:.1f} minutes")

# Parse results
scores = {"escalation": {}, "blame": {}, "cta": {}}
parse_errors = []

# Retrieve the final batch status
batch_final = client.messages.batches.retrieve(batch.id)

if batch_final.results_url:
    print(f"📥 Fetching results from batch {batch.id}")
    
    headers = {
        "x-api-key": os.getenv("ANTHROPIC_API_KEY"),
        "anthropic-version": "2023-06-01"
    }
    
    response = requests.get(batch_final.results_url, headers=headers, stream=True)
    
    if response.status_code == 200:
        # Process JSONL results line by line
        for line in response.iter_lines():
            if not line:
                continue
                
            try:
                result = json.loads(line)
                custom_id = result.get("custom_id")
                
                if custom_id is None:
                    continue
                
                idx = int(custom_id)
                
                # Check if request succeeded
                if result.get("result", {}).get("type") != "succeeded":
                    parse_errors.append(f"Request {custom_id} failed: {result.get('result', {}).get('type')}")
                    continue
                
                # Extract the response text
                message_content = result["result"]["message"]["content"][0]["text"]
                
                # Parse the three integers - more flexible parsing
                # Remove any whitespace and look for pattern like "E,B,C"
                clean_content = message_content.strip()
                match = re.match(r'^(\d+),(-?\d+),(\d+)', clean_content)
                
                if match:
                    e, b, c = int(match.group(1)), int(match.group(2)), int(match.group(3))
                    
                    # Validate and store scores
                    if 0 <= e <= 10: 
                        scores["escalation"][idx] = e
                    else:
                        parse_errors.append(f"Invalid escalation score {e} for request {custom_id}")
                        
                    if b in (-1, 0, 1): 
                        scores["blame"][idx] = b
                    else:
                        parse_errors.append(f"Invalid blame score {b} for request {custom_id}")
                        
                    if c in (0, 1): 
                        scores["cta"][idx] = c
                    else:
                        parse_errors.append(f"Invalid CTA score {c} for request {custom_id}")
                else:
                    parse_errors.append(f"Could not parse response for request {custom_id}: {message_content}")
                    
            except Exception as e:
                parse_errors.append(f"Error parsing result: {e}")
                continue
                
        print(f"✅ Successfully parsed {len(scores['escalation'])} results")
    else:
        print(f"❌ Error fetching results: HTTP {response.status_code}")
else:
    print("❌ No results URL available")

# Map scores back to dataframe
df["escalation_score"] = df["batch_idx"].map(scores["escalation"]).astype("Int64")
df["blame_direction"] = df["batch_idx"].map(scores["blame"]).astype("Int64")
df["has_cta"] = df["batch_idx"].map(scores["cta"]).astype("Int64")

# Save results
df_out = df.drop(columns=["batch_idx"])
df_out.to_csv(OUT_CSV, index=False)

# Summary statistics
print(f"\n✅ Scoring complete")
print(f"   Total posts: {len(df)}")
print(f"   Successfully scored: {len(scores['escalation'])}")
print(f"   Failed: {len(df) - len(scores['escalation'])}")
print(f"   Success rate: {len(scores['escalation'])/len(df)*100:.1f}%")

if parse_errors:
    print(f"\n⚠️  Parse errors encountered ({len(parse_errors)} total):")
    for error in parse_errors[:5]:
        print(f"   - {error}")
    if len(parse_errors) > 5:
        print(f"   ... and {len(parse_errors) - 5} more errors")

# Show distribution of scores
if len(scores['escalation']) > 0:
    print("\n📊 Score distributions:")
    esc_df = pd.Series(scores['escalation'].values())
    print(f"   Escalation: mean={esc_df.mean():.1f}, std={esc_df.std():.1f}, range={esc_df.min()}-{esc_df.max()}")
    
    blame_df = pd.Series(scores['blame'].values())
    blame_counts = blame_df.value_counts().sort_index()
    print(f"   Blame direction: {dict(blame_counts)}")
    
    cta_df = pd.Series(scores['cta'].values())
    print(f"   Call-to-action: {cta_df.sum()} posts ({cta_df.mean()*100:.1f}%) have CTAs")

print(f"\n📁 Results saved to: {OUT_CSV}")

## Score with OpenAI Model of Choice

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════╗
# ║  SCORE Truth Social posts - OpenAI Batch API Version                  ║
# ╚══════════════════════════════════════════════════════════════════════╝
from pathlib import Path
import json, time, pandas as pd, re
from openai import OpenAI
from dotenv import load_dotenv
import os
import requests
from datetime import datetime

load_dotenv()

# Configuration
ROOT = Path.cwd().resolve().parents[0]
TRUTH_CSV = ROOT / "outputs" / "truth_matches_merged.csv"
OUT_CSV = ROOT / "outputs" / "truth_scores_openai_gpt-4o-mini.csv"
MODEL = "gpt-4o-mini"  # Options: gpt-4o-mini, gpt-4o, gpt-3.5-turbo

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ── Δ CODE: add right after you create the OpenAI client ────────────────
RUBRIC_PATH = ROOT / "prompt_files" / "truth_rubric.txt"
RUBRIC_PATH.write_text(SYSTEM_PROMPT)        # save the long prompt to disk

rubric_file = client.files.create(
    file=open(RUBRIC_PATH, "rb"),
    purpose="assistants"                     # file can be injected into messages
)

print(f"✅ Rubric file uploaded: {rubric_file.id}")
# ─────────────────────────────────────────────────────────────────────────


# Load data
df = pd.read_csv(TRUTH_CSV)
print(f"📊 Processing {len(df)} Truth Social posts")

# Uncomment for dry run
df = df.head(5).copy()
print(f"🧪 DRY RUN WITH {len(df)} POSTS")

# Add index for tracking
df["batch_idx"] = range(len(df))

# Step 1: Create JSONL file with requests
print("📝 Creating batch input file...")
batch_requests = []

for idx, row in df.iterrows():
    if pd.isna(row.get("text")) or str(row["text"]).strip() == "":
        continue
    
    system_prompt = (
    "You are a scoring assistant. "
    "Consult the attached rubric and reply ONLY as E,B,C.\n\n"
    f"<file:{rubric_file.id}>"
    )
    
    request = {
        "custom_id": f"request-{row['batch_idx']}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": MODEL,
            "messages": [
                { "role": "system", "content": system_prompt },
                { "role": "user",   "content": str(row['text'])[:1500] }
            ],
            "max_tokens": 10,
            "temperature": 0
        }
    }
    batch_requests.append(request)



# Write JSONL file
input_file_path = ROOT / "outputs" / "openai_batch_input.jsonl"
with open(input_file_path, 'w') as f:
    for request in batch_requests:
        f.write(json.dumps(request) + '\n')

print(f"✅ Created batch input file with {len(batch_requests)} requests")

# Step 2: Upload the file
print("📤 Uploading batch file to OpenAI...")
with open(input_file_path, 'rb') as f:
    batch_input_file = client.files.create(
        file=f,
        purpose="batch"
    )

print(f"✅ File uploaded: {batch_input_file.id}")

# Step 3: Create the batch
print("🚀 Creating batch job...")
batch = client.batches.create(
    input_file_id=batch_input_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "Truth Social Ukraine war posts scoring",
        "timestamp": datetime.now().isoformat()
    }
)

print(f"✅ Batch created: {batch.id}")
print(f"   Status: {batch.status}")

# Step 4: Monitor the batch
print("\n⏳ Monitoring batch progress...")
start_time = time.time()

while batch.status not in ["completed", "failed", "expired", "cancelled"]:
    time.sleep(30)  # Check every 30 seconds
    batch = client.batches.retrieve(batch.id)
    
    elapsed = time.time() - start_time
    total = batch.request_counts.total
    completed = batch.request_counts.completed
    failed = batch.request_counts.failed
    
    if total > 0:
        progress = (completed + failed) / total * 100
        print(f"   Progress: {progress:.1f}% ({completed} completed, {failed} failed) - {elapsed/60:.1f} minutes elapsed")
    else:
        print(f"   Status: {batch.status} - {elapsed/60:.1f} minutes elapsed")

print(f"\n✅ Batch {batch.status} in {(time.time() - start_time)/60:.1f} minutes")

# Step 5: Retrieve and parse results
if batch.status == "completed" and batch.output_file_id:
    print("📥 Downloading results...")
    
    # Download the output file
    output_file = client.files.content(batch.output_file_id)
    output_file_path = ROOT / "outputs" / "openai_batch_output.jsonl"
    
    with open(output_file_path, 'wb') as f:
        f.write(output_file.content)
    
    print("✅ Results downloaded")
    
    # Parse results
    scores = {"escalation": {}, "blame": {}, "cta": {}}
    parse_errors = []
    
    with open(output_file_path, 'r') as f:
        for line in f:
            if not line.strip():
                continue
                
            try:
                result = json.loads(line)
                custom_id = result.get("custom_id", "")
                
                # Extract index from custom_id (format: "request-123")
                if custom_id.startswith("request-"):
                    idx = int(custom_id.split("-")[1])
                else:
                    parse_errors.append(f"Invalid custom_id format: {custom_id}")
                    continue
                
                # Check if request succeeded
                if result.get("response", {}).get("status_code") != 200:
                    parse_errors.append(f"Request {custom_id} failed with status {result.get('response', {}).get('status_code')}")
                    continue
                
                # Extract the response content
                response_body = result["response"]["body"]
                choices = response_body.get("choices", [])
                
                if not choices:
                    parse_errors.append(f"No choices in response for {custom_id}")
                    continue
                
                message_content = choices[0]["message"]["content"]
                
                # Parse the three integers
                clean_content = message_content.strip()
                match = re.match(r'^(\d+),(-?\d+),(\d+)', clean_content)
                
                if match:
                    e, b, c = int(match.group(1)), int(match.group(2)), int(match.group(3))
                    
                    # Validate and store scores
                    if 0 <= e <= 10:
                        scores["escalation"][idx] = e
                    else:
                        parse_errors.append(f"Invalid escalation score {e} for {custom_id}")
                    
                    if b in (-1, 0, 1):
                        scores["blame"][idx] = b
                    else:
                        parse_errors.append(f"Invalid blame score {b} for {custom_id}")
                    
                    if c in (0, 1):
                        scores["cta"][idx] = c
                    else:
                        parse_errors.append(f"Invalid CTA score {c} for {custom_id}")
                else:
                    parse_errors.append(f"Could not parse response for {custom_id}: {message_content[:100]}")
                    
            except Exception as e:
                parse_errors.append(f"Error parsing result: {str(e)}")
                continue
    
    print(f"✅ Successfully parsed {len(scores['escalation'])} results")
    
    # Map scores back to dataframe
    df["escalation_score"] = df["batch_idx"].map(scores["escalation"]).astype("Int64")
    df["blame_direction"] = df["batch_idx"].map(scores["blame"]).astype("Int64")
    df["has_cta"] = df["batch_idx"].map(scores["cta"]).astype("Int64")
    
    # Save results
    df_out = df.drop(columns=["batch_idx"])
    df_out.to_csv(OUT_CSV, index=False)
    
    # Summary statistics
    print(f"\n✅ Scoring complete")
    print(f"   Total posts: {len(df)}")
    print(f"   Successfully scored: {len(scores['escalation'])}")
    print(f"   Failed: {len(df) - len(scores['escalation'])}")
    print(f"   Success rate: {len(scores['escalation'])/len(df)*100:.1f}%")
    
    if parse_errors:
        print(f"\n⚠️  Parse errors encountered ({len(parse_errors)} total):")
        for error in parse_errors[:5]:
            print(f"   - {error}")
        if len(parse_errors) > 5:
            print(f"   ... and {len(parse_errors) - 5} more errors")
    
    # Show distribution of scores
    if len(scores['escalation']) > 0:
        print("\n📊 Score distributions:")
        esc_df = pd.Series(scores['escalation'].values())
        print(f"   Escalation: mean={esc_df.mean():.1f}, std={esc_df.std():.1f}, range={esc_df.min()}-{esc_df.max()}")
        
        blame_df = pd.Series(scores['blame'].values())
        blame_counts = blame_df.value_counts().sort_index()
        print(f"   Blame direction: {dict(blame_counts)}")
        
        cta_df = pd.Series(scores['cta'].values())
        print(f"   Call-to-action: {cta_df.sum()} posts ({cta_df.mean()*100:.1f}%) have CTAs")
    
    print(f"\n📁 Results saved to: {OUT_CSV}")
    
    # Clean up temporary files (optional)
    # input_file_path.unlink()
    # output_file_path.unlink()
    
else:
    print(f"❌ Batch failed or no output file available")
    if hasattr(batch, 'errors') and batch.errors:
        print(f"   Errors: {batch.errors}")

In [ ]:
openai batches list --limit 10

In [ ]:
bad = client.batches.retrieve("batch_683f2561d5908190932781bfec5b94b6")
print(bad.errors[0]["message"])      # shows 400: ‘content_block.type must be one of (“text”, “image_url”)’

